In [2]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import math
import numpy as np
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aaditya11/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# def docPreProcessing(filepath):
#     stopwords_dict = Counter(stopwords.words('english'))

#     file = open(filepath, 'r', errors='replace')
#     lines = file.readlines()
    
#     # print(filepath)

#     stopwords_dict = Counter(stopwords.words('english'))
    
    
#     doctext = ""
#     for line in lines:
#         line = line.translate(str.maketrans('', '', string.punctuation)).strip().lower()
#         line = ' '.join([word for word in line.split() if word not in stopwords_dict])
#         doctext += line
        
#     doctextList = doctext.split()
#     return doctextList

In [15]:
#this is wrong, check cpp implementation

def docPreProcessing(filepath):
    stopwords_dict = Counter(stopwords.words('english'))


    docTextList = []
    with open(filepath, 'r', errors='replace') as file:
        for line in file:
            for word in line.split():
                word = word.translate(str.maketrans('', '', string.punctuation)).strip().lower()
                if word not in stopwords_dict and word != '':
                    docTextList.append(PorterStemmer().stem(word))

    return docTextList

['bat',
 'bat',
 'flip',
 'flipper',
 'flip',
 'batter',
 'bring',
 'batteri',
 'bringer',
 'bring',
 'flop']

In [13]:
import copy

ps = PorterStemmer()
wordlist = docPreProcessing(r'../testset/sci.space/60151')
wordlistcopy = copy.copy(wordlist)

wordlistcopy = list(map(ps.stem, wordlistcopy))

for word in wordlistcopy:
    print(word)

# docPreProcessing(r'../testset/sci.space/59497')

henryzootorontoedu
henri
spencer
subject
hlv
fred
prefab
space
station
articl
c5133agzxnewscsouiucedu
jbh55289uxacsouiucedu
josh
hopkin
write
titan
iv
launch
aint
cheap
grant
that
titan
iv
bought
governemnt
titan
iii
actual
cheapest
way
put
pound
space
us
expend
launcher
case
rather
iron
poorli
commerci
market
singl
titan
iii
order
problem
commerci
titan
mm
made
littl
attempt
market
theyr
basic
happi
govern
busi
dont
want
learn
sell
commerci
secondari
problem
bit
big
theyd
need
go
multisatellit
launch
la
arian
complic
market
task
quit
significantli
also
problem
launch
facil
wrong
time
get
start
properli
memori
serv
pad
use
mar
observ
launch
come
heavi
refurbish
work
prevent
launch
year
ct
launch
mar
observ
one
strand
intelsat
least
one
brother
reach
orbit
properli
work
one
man
work
henri
spencer
u
toronto
zoolog
kipl
henryzootorontoedu
utzoohenri


In [12]:
l = ['bat', 'batting', 'flip', 'flipped', 'flipping']

psl = map(PorterStemmer().stem, l)

print(list(psl))

['bat', 'bat', 'flip', 'flip', 'flip']


In [4]:
#replace with cpp implementation, 40% reduction in time

def tf(wordlist):

    wordmap = {}
    
    for word in wordlist:
        if word in wordmap:
            wordmap[word] += 1
        else:
            wordmap[word] = 1
                
    return wordmap
        


In [ ]:
tf(docPreProcessing(r'pstest.txt'))

In [7]:
import time

starttime = time.process_time()

wordmap = tf(docPreProcessing(r'../testset/sci.space/60151'))



# open a file and write wordmap to it
with open('wordmap.txt', 'w') as file:
    for word in wordmap:
        file.write(word + " " + str(wordmap[word]) + "\n")

print("python time: ", (time.process_time() - starttime) * 1000000)

python time:  9361.651999999942


In [5]:
# def tfidfMapBuilder(doclist):
#     # termset = set()

#     # for doc in doclist:
#         # termset.update(set(docPreProcessing(doc)))

#     # df_columns = doclist.copy()
#     df_columns = []
#     df_columns.append('df')
#     df_columns.extend(doclist)
#     # print(df_columns)

#     df = pd.DataFrame(data=0, index = [], columns=df_columns)

#     for doc in doclist:
#         wordmap = tf(docPreProcessing(doc))
#         for term in wordmap:
#             if term not in df.index:
#                 new_row = pd.DataFrame(data=0, index=[term], columns=df_columns)
#                 df = pd.concat([df, new_row])
#             df.loc[term, doc] = wordmap[term]
#             df.loc[term, 'df'] += 1

#     # print(df)

#     # N = len(doclist)
#     # df['df'] = df['df'].apply(lambda x: math.log(N/x))

#     return df

#     # print(len(termset))

In [6]:

def tfidfMapBuilder(doclist):
    termset = set()

    for doc in doclist:
        termset.update(set(tf(docPreProcessing(doc)).keys()))

    df_columns = doclist.copy()
    df_columns.append('df')

    df = pd.DataFrame(0, index=list(termset), columns=df_columns)

    for doc in doclist:
        wordmap = tf(docPreProcessing(doc))
        for term in wordmap:
            df.at[term, doc] = wordmap[term]
            df.at[term, 'df'] += 1

    N = len(doclist)

    return df

In [7]:
def mapToMatrix(df, N):
    df['df'] = df['df'].apply(lambda x: math.log(N/x))

    for index, row in df.iterrows():
        term_df = row['df']
        row = row.apply(lambda x: x * term_df)
    
    df = df.drop(columns=['df'], axis=1)

    return df

In [8]:
def cosineSimilarityScore(q, D):
    q_mag = np.sqrt(q.dot(q))
    
    qT = np.reshape(q, (1, q.shape[0]))
    qTD = np.matmul(qT, D).reshape((D.shape[1], ))
    
    D_mags = np.sqrt(np.sum(D*D, axis=0))
    divisors = q_mag * D_mags
    cos_thetas = np.divide(qTD, divisors)
    
    cos_thetas = np.clip(cos_thetas, -1, 1)
    
    scores = np.arccos(cos_thetas)
    
    return scores
    

In [9]:
def fileCollector(path):
    filelist = []
    
    for (root, dirs, files) in os.walk(path, topdown=True):
        for file in files:
            filelist.append(os.path.join(root, file))
            
            if len(filelist) == 100:
                return filelist

    return filelist

In [10]:
doclist = []

doclist.append('query.txt')

doclist.extend(fileCollector(r'../testset/sci.space'))

tfidfDataFrame = mapToMatrix(tfidfMapBuilder(doclist), len(doclist))

# tfidfDataFrame
# tfidfMapBuilder.to_csv('tfidf.csv')

# tfidfMap = tfidfMapBuilder(doclist)

# import sqlite3
# database = "../example.sqlite"
# conn = sqlite3.connect(database)
# tfidfMap.to_sql(name='Users', con=conn, if_exists='replace', index=False)
# conn.close()

# print("saved to database!!!!!!!!!!!!!!!!")
# 
# cnx = sqlite3.connect(database)
# tfidfMap = pd.read_sql_query("SELECT * FROM Users", cnx)

# print(tfidfMap.to_numpy())

# tfidfDataFrame = mapToMatrix(tfidfMap, len(doclist))


# # tfidfMap.to_sql('tfidf', con=engine, if_exists='replace')


doc_tfidfMatrix = tfidfDataFrame.loc[:, tfidfDataFrame.columns != 'query.txt'].to_numpy().round(decimals=4)
query_vector = tfidfDataFrame.loc[:, "query.txt"].to_numpy().round(decimals=4)

['query.txt', '../testset/sci.space/60777', '../testset/sci.space/61312', '../testset/sci.space/61023', '../testset/sci.space/61090', '../testset/sci.space/61145', '../testset/sci.space/61245', '../testset/sci.space/61042', '../testset/sci.space/61433', '../testset/sci.space/61064', '../testset/sci.space/60978', '../testset/sci.space/60894', '../testset/sci.space/60910', '../testset/sci.space/60785', '../testset/sci.space/60252', '../testset/sci.space/61035', '../testset/sci.space/60888', '../testset/sci.space/61186', '../testset/sci.space/60253', '../testset/sci.space/61138', '../testset/sci.space/62477', '../testset/sci.space/60808', '../testset/sci.space/61157', '../testset/sci.space/61492', '../testset/sci.space/61515', '../testset/sci.space/60845', '../testset/sci.space/60896', '../testset/sci.space/61445', '../testset/sci.space/61036', '../testset/sci.space/61551', '../testset/sci.space/61320', '../testset/sci.space/60916', '../testset/sci.space/60170', '../testset/sci.space/6242

In [11]:
scorelist = cosineSimilarityScore(query_vector, doc_tfidfMatrix)
scoremap = {}

for index, score in enumerate(scorelist):
    scoremap[tfidfDataFrame.columns[index + 1]] = score

scoremap = dict(sorted(scoremap.items(), key=lambda item:item[1]))

for doc in list(scoremap.items())[:5]:
    print(doc[0] + " : " + str(doc[1]))


../testset/sci.space/60827 : 1.4725654859548984
../testset/sci.space/61138 : 1.473050747060915
../testset/sci.space/61288 : 1.512040504079174
../testset/sci.space/60168 : 1.5331981328975341
../testset/sci.space/61339 : 1.5389541866018606


In [36]:
doclist = []

doclist.append('query.txt')

doclist.extend(fileCollector(r'../testset/sci.space'))

print(doclist)

['query.txt', '../testset/sci.space/60777', '../testset/sci.space/61312', '../testset/sci.space/61023', '../testset/sci.space/61090', '../testset/sci.space/61145', '../testset/sci.space/61245', '../testset/sci.space/61042', '../testset/sci.space/61433', '../testset/sci.space/61064', '../testset/sci.space/60978', '../testset/sci.space/60894', '../testset/sci.space/60910', '../testset/sci.space/60785', '../testset/sci.space/60252', '../testset/sci.space/61035', '../testset/sci.space/60888', '../testset/sci.space/61186', '../testset/sci.space/60253', '../testset/sci.space/61138', '../testset/sci.space/62477', '../testset/sci.space/60808', '../testset/sci.space/61157', '../testset/sci.space/61492', '../testset/sci.space/61515', '../testset/sci.space/60845', '../testset/sci.space/60896', '../testset/sci.space/61445', '../testset/sci.space/61036', '../testset/sci.space/61551', '../testset/sci.space/61320', '../testset/sci.space/60916', '../testset/sci.space/60170', '../testset/sci.space/6242